In [10]:
import numpy as np
from implementations_leo import *
from proj1_helpers import *
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
#Load the data both from train and test csv file
y, x, ids = load_csv_data("train.csv", sub_sample=False)
#_, x_submission, ids_submission = load_csv_data("test.csv", sub_sample=False)

In [11]:
a = [3,2]
b = [7,8]
print(np.multiply(a,b))

[21 16]


In [4]:
bad_sub_arrays = np.where(x == -999.)[0] #find lines of the matrix x where we have at least 
print('With repeated lines because -999 appears more than once in some lines',bad_sub_arrays[:50]) # just the beginning
print('\ntruncated output', y[:20])
bad_sub_arrays = np.unique(bad_sub_arrays)
print('The list of lines with insignificant data, with no repetitions')
bad_sub_arrays[:50]

xclean = np.delete(x, bad_sub_arrays, axis = 0)
print('There are no -999 anymore',np.where(xclean == -999.)) #it works!

xclean = standardize(xclean)
print('\n Here is a bit of the clean x\n', xclean[0:10]) #works

yclean = np.delete(y, bad_sub_arrays, axis = 0)
print('\nHere is a bit of the clean y', yclean[:10]) #appears to work. Compare with y above

With repeated lines because -999 appears more than once in some lines [1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 7 7
 7 7 7 7 7 8 8 8 8 8 8 8 8]

truncated output [ 1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.
 -1. -1.]
The list of lines with insignificant data, with no repetitions
There are no -999 anymore (array([], dtype=int64), array([], dtype=int64))

 Here is a bit of the clean x
 [[ 0.25767126  0.53283515  0.47461094 -1.09345441 -0.87064819 -0.63242833
   0.98731346  1.37804494  0.51549536 -0.64313461  0.09205652  0.8203249
  -0.6621994  -0.44063129  0.86513362  0.20892619 -0.02803712  1.90142579
  -1.352776   -0.89329993 -0.15656359 -0.54298155 -0.6604439  -0.57866597
   1.23439906  0.25179525 -0.36784371  0.61296467 -1.35932881 -0.64515168]
 [-0.54438812 -0.70635039 -0.47504382  0.06438322  0.11525063 -0.233525
   0.09492308 -0.81913195  1.21210996 -0.02033658 -0.81324448 -0.17116673
   1.28298905  0.26731433  0.31741279  0

In [ ]:
def getridof999(x):
    for col in range(x.shape[1]):
        x1 = 1*x
        x1[x1==-999] = 0
        indices = [i for i,item in enumerate(x[:,col] == -999) if item]
        length = len(x1[:,col])
        nonzeros = sum(x1[:,col] != 0)
        mean = np.mean(x1[:,col])*length/nonzeros
        norm = np.linalg.norm(x1[:,col])/nonzeros
        for i in indices:
            x[i,col] = np.random.normal(mean, norm)
    return x

In [16]:
def getridof999_2(x):
    ids = []
    for col in range(x.shape[1]):
        ids.append(np.where(x[:,col] == -999)[0])
    ids = np.unique(ids)
    print(ids[:30])
    x = x[ids, :]
    print(x[:10,:10])
    return x
    

In [7]:
'''Define values to perform Ridge regression with simple splitting of the dataset'''

degree = 12
ratio = 0.70 #train/test examples
lambdas = np.logspace(-6,0,10)
seed = 52

w = ridge_with_simple_splitting(y,x, degree, ratio, lambdas, seed)

MemoryError: 

In [5]:
def ridge_with_simple_splitting(y,x, degree, ratio, lambdas, seed = 1):
    loss = []
    y_train, x_train, y_test, x_test = split_data(y, x, ratio, seed)

    phi_test = build_poly(x_test, degree, False)
    phi_train = build_poly(x_train, degree, False)

    for lambda_ in lambdas:

        w = ridge_regression(y_train, phi_train, lambda_)
        rmse_test = cost_function(y_test, phi_test, w)

        loss.append(rmse_test)

        print("Correct answers: ",predict(y_test,phi_test,w), '%', "for lambda = %f" %lambda_)
    semilog_loss_lambda_plot(loss, lambdas, seed, degree)
    return w

In [8]:
def cross_validation_ridge(y, x, k_fold, degree, lambdas, seed = 1):
    '''perform cross validation on ridge regression
    lambdas: array, better if log spaced
    print percentage of correct answers for each k, lambda
    plot in semilog scale rmse as function of lambda'''
    
    #create empty
    loss = np.zeros((len(lambdas))) 
    #add poly values to the features
    phi = build_poly(x, degree, False)
    #build indices for cross validation
    k_indices = build_k_indices(y, k_fold, seed)

    for k in range(k_fold):
        #split data according to kth fold
        y_train, phi_train, y_test, phi_test = split_data_cross(y, phi, k, k_indices, degree)

        loss_temp = [] #empty list to store losses for a given k
        
        for lambda_ in lambdas:
            w = ridge_regression(y_train, phi_train, lambda_)
            
            rmse_test = cost_function(y_test, phi_test, w)/k_fold #divide by k_fold in order to mean over them
            loss_temp.append(rmse_test)

            print("Correct answers: ",predict(y_test,phi_test,w), '%', "for k = %i, lambda = %f" %(k, lambda_))
        loss += loss_temp
    semilog_loss_lambda_plot(loss, lambdas, seed, degree)

In [9]:
k_fold = 4
degree = 12
lambdas = np.logspace(-5,1,10)
seed = 333

cross_validation_ridge(y, x, k_fold, degree, lambdas, seed)

MemoryError: 

In [ ]:
def create_submission(x_submission, degree, ids_submission):
    phi_submission = build_poly(x_submission, degree) 

    y_predicted = np.sign(phi_submission.dot(w))
    create_csv_submission(ids_submission, y_predicted, "predictions_7.csv")

In [ ]:
#create_submission(x_submission, degree, ids_submission)
phi_submission = build_poly(x_submission, degree) 

y_predicted = np.sign(phi_submission.dot(w))
create_csv_submission(ids_submission, y_predicted, "predictions_7.csv")

In [ ]:
sum(y_predicted == -1)/len(y_predicted)